

# Import nesaccry libraries and read the data



In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import os
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from tensorflow.keras.utils import to_categorical
import torch.nn as nn
import torch.optim as optim
from transformers import AdamW
from transformers import AutoTokenizer, BertTokenizer, BertModel
import torch
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, AdamW
import torch
import torch.nn as nn
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional,GRU,SimpleRNN
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from transformers import BertTokenizer, BertForSequenceClassification, BertModel
from sklearn.linear_model import LogisticRegression

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)


Using device: cpu


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_train=pd.read_excel("/content/drive/MyDrive/NLP files-20240522T143610Z-001/NLP files/preprocessed_data_train2.xlsx")
df_test=pd.read_excel("/content/drive/MyDrive/NLP files-20240522T143610Z-001/NLP files/preprocessed_data_test2.xlsx")

In [ ]:
df_train.head()

,Type,News,processed_text
0,politics,اشتباك الحريري-عون.. اتهامات لباسيل بالتمسك با...,اشتباك حريري عون اتهام باسيل تمسك ثلث معطل بقا...
1,politics,عون: الحريري أصبح غريب الأطوار,عون حريري أصبح غريب اطوار
2,politics,وزير الخارجية الأمريكي: ندرس سحب كامل قواتنا م...,وزير خارجيه أمريكي درس سحب كامل قوة أفغانستان حل
3,politics,أفغانستان.. استعدادات حثيثة لاجتماع تركيا وكاب...,أفغانستان استعداد حثيث اجتماع تركيا كابل اتهم ...
4,politics,أندبندنت‮:‬ ‬مفاوضات ‬سرّية ‬‬كادت ‬تنقذ ‬القذافي,اندبندنت مفاوضة سر يه كاد أنقذ قذافي


In [ ]:
train_df, val_df = train_test_split(df_train, test_size=0.2, random_state=0)

In [ ]:
tokenizer = Tokenizer(oov_token='<oov>') # For those words which are not found in word_index
all_text=df_train['processed_text'].tolist()
tokenizer.fit_on_texts(all_text)
total_words = len(tokenizer.word_index) + 1

print("Total number of words: ", total_words)
print("Word: ID")
print("------------")
print("<oov>: ", tokenizer.word_index['<oov>'])
print("غريب: ", tokenizer.word_index['غريب'])
print("كره: ", tokenizer.word_index['كره'])
print("اطوار: ", tokenizer.word_index['اطوار'])

Total number of words:  5972
Word: ID
------------
<oov>:  1
غريب:  2076
كره:  65
اطوار:  3539


In [ ]:
train_sequences = tokenizer.texts_to_sequences(train_df['processed_text'])
val_sequences = tokenizer.texts_to_sequences(val_df['processed_text'])
test_sequences = tokenizer.texts_to_sequences(df_test['processed_text'])

In [ ]:
max_sequence_len = max(len(seq) for seq in train_sequences)

In [ ]:
padded_train = pad_sequences(train_sequences, maxlen=max_sequence_len, padding='pre')
padded_val = pad_sequences(val_sequences, maxlen=max_sequence_len, padding='pre',truncating='pre')
padded_test = pad_sequences(test_sequences, maxlen=max_sequence_len, padding='pre',truncating='pre')

In [ ]:
y_train = train_df['Type']
y_val=val_df['Type']
y_test=df_test['Type']

In [ ]:
label=LabelEncoder()

y_train_encoded=label.fit_transform(y_train)
y_val_encoded=label.transform(y_val)
y_test_encoded=label.transform(y_test)

In [ ]:
y_train_one_hot_encoded=to_categorical(y_train_encoded)
y_val_one_hot_encoded=to_categorical(y_val_encoded)
y_test_one_hot_encoded=to_categorical(y_test_encoded)

# BERT

In [ ]:
bert_results={}

In [ ]:
def get_BERT_embeddings(model_name,text):
  embedding_list=[]
  tokenizer=BertTokenizer.from_pretrained(model_name)
  model=BertModel.from_pretrained(model_name)
  model
  for inputs in text:
    tokens=tokenizer(inputs,padding=True,truncation=True,return_tensors='pt')
    inputs_ids=tokens['input_ids']
    attention_mask=tokens['attention_mask']
    with torch.no_grad():
      outputs=model(inputs_ids,attention_mask=attention_mask)
      #extracting the embeddings
      embeddings=outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
      embedding_list.append(embeddings)
  embeddings_np=np.array(embedding_list)

  print("embeddings are done extracting with a shape of ",embeddings_np.shape)
  return embeddings_np

In [ ]:
model_name = "aubmindlab/bert-base-arabertv02"
train_embeddings_bert=get_BERT_embeddings(model_name,train_df['processed_text'])
val_embeddings_bert=get_BERT_embeddings(model_name,val_df['processed_text'])
test_embeddings_bert=get_BERT_embeddings(model_name,df_test['processed_text'])

embeddings are done extracting with a shape of  (4000, 768)
embeddings are done extracting with a shape of  (1000, 768)
embeddings are done extracting with a shape of  (1000, 768)


## Traditional models

### Naive Bayes

In [ ]:
clf_nb_BERT = GaussianNB()
clf_nb_BERT.fit(train_embeddings_bert, y_train)

y_pred_clf_nb_bert_val = clf_nb_BERT.predict(val_embeddings_bert)

accuracy_clf_nb_bert_val = accuracy_score(y_val,y_pred_clf_nb_bert_val)
print(f"validation Accuracy: {accuracy_clf_nb_bert_val}")

precision_clf_nb_bert_val = precision_score(y_val, y_pred_clf_nb_bert_val, average='weighted')
recall_clf_nb_bert_val = recall_score(y_val,y_pred_clf_nb_bert_val, average='weighted')
f1_clf_nb_bert_val = f1_score(y_val,y_pred_clf_nb_bert_val, average='weighted')

print(f'Accuracy: {accuracy_clf_nb_bert_val:.4f}')
print(f'Precision: {precision_clf_nb_bert_val:.4f}')
print(f'Recall: {recall_clf_nb_bert_val:.4f}')
print(f'F1 Score: {f1_clf_nb_bert_val:.4f}')

classification_report_val=classification_report(y_val,y_pred_clf_nb_bert_val)
print(classification_report_val)

bert_results['Naive Bayes Accuracy BERT val']=accuracy_clf_nb_bert_val
bert_results['Naive Bayes Precision BERT val']=precision_clf_nb_bert_val
bert_results['Naive Bayes Recall BERT val']=recall_clf_nb_bert_val
bert_results['Naive Bayes F1 BERT val']=f1_clf_nb_bert_val

validation Accuracy: 0.916
Accuracy: 0.9160
Precision: 0.9229
Recall: 0.9160
F1 Score: 0.9180
              precision    recall  f1-score   support

    economic       0.89      0.80      0.84       180
    politics       0.94      0.94      0.94       601
       sport       0.96      0.95      0.96       198
        tech       0.46      0.81      0.59        21

    accuracy                           0.92      1000
   macro avg       0.81      0.88      0.83      1000
weighted avg       0.92      0.92      0.92      1000



In [ ]:
y_test

0      politics
1      politics
2      politics
3      politics
4      politics
         ...   
995        tech
996        tech
997        tech
998        tech
999        tech
Name: Type, Length: 1000, dtype: object

In [ ]:

y_pred_clf_nb_bert_test = clf_nb_BERT.predict(test_embeddings_bert)


accuracy_clf_nb_bert_test = accuracy_score(y_test,y_pred_clf_nb_bert_test)
precision_clf_nb_bert_test = precision_score(y_test,y_pred_clf_nb_bert_test, average='weighted')
recall_clf_nb_bert_test = recall_score(y_test, y_pred_clf_nb_bert_test, average='weighted')
f1_clf_nb_bert_test = f1_score(y_test, y_pred_clf_nb_bert_test, average='weighted')
print(f"test Accuracy: {accuracy_clf_nb_bert_test}")
print(f"Precision for test: {precision_clf_nb_bert_test:.2f}")
print(f"Recall for test: {recall_clf_nb_bert_test:.2f}")
print(f"F1 Score for test: {f1_clf_nb_bert_test:.2f}")
classification_report_test=classification_report(y_test,y_pred_clf_nb_bert_test)
print(classification_report_test)

bert_results['Naive Bayes Accuracy BERT test']=accuracy_clf_nb_bert_test
bert_results['Naive Bayes Precision BERT test']=precision_clf_nb_bert_test
bert_results['Naive Bayes Recall BERT test']=recall_clf_nb_bert_test
bert_results['Naive Bayes F1 BERT test']=f1_clf_nb_bert_test

test Accuracy: 0.893
Precision for test: 0.89
Recall for test: 0.89
F1 Score for test: 0.89
              precision    recall  f1-score   support

    economic       0.84      0.74      0.79       200
    politics       0.89      0.93      0.91       512
       sport       0.98      0.96      0.97       200
        tech       0.84      0.86      0.85        88

    accuracy                           0.89      1000
   macro avg       0.89      0.87      0.88      1000
weighted avg       0.89      0.89      0.89      1000



### Logestic Regression

In [ ]:
clf_lr_BERT = LogisticRegression()
clf_lr_BERT.fit(train_embeddings_bert, y_train)

y_pred_clf_lr_bert_val = clf_lr_BERT.predict(val_embeddings_bert)

accuracy_clf_lr_bert_val = accuracy_score(y_val,y_pred_clf_lr_bert_val)
precision_clf_lr_bert_val = precision_score(y_val,y_pred_clf_lr_bert_val, average='weighted')
recall_clf_lr_bert_val = recall_score(y_val, y_pred_clf_lr_bert_val, average='weighted')
f1_clf_lr_bert_val = f1_score(y_val, y_pred_clf_lr_bert_val, average='weighted')
print(f"validation Accuracy: {accuracy_clf_lr_bert_val}")
print(f"Precision for validation: {precision_clf_lr_bert_val:.2f}")
print(f"Recall for validation: {recall_clf_lr_bert_val:.2f}")
print(f"F1 Score for validation: {f1_clf_lr_bert_val:.2f}")
classification_report_val=classification_report(y_val,y_pred_clf_lr_bert_val)
print(classification_report_val)

bert_results['Logestic Regression Accuracy BERT val']=accuracy_clf_lr_bert_val
bert_results['Logestic Regression Precision BERT val']=precision_clf_lr_bert_val
bert_results['Logestic Regression Recall BERT val']=recall_clf_lr_bert_val
bert_results['Logistic Regression F1 BERT val']=f1_clf_lr_bert_val


validation Accuracy: 0.922
Precision for validation: 0.92
Recall for validation: 0.92
F1 Score for validation: 0.92
              precision    recall  f1-score   support

    economic       0.89      0.79      0.84       180
    politics       0.94      0.95      0.94       601
       sport       0.94      0.96      0.95       198
        tech       0.62      0.71      0.67        21

    accuracy                           0.92      1000
   macro avg       0.85      0.86      0.85      1000
weighted avg       0.92      0.92      0.92      1000



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
y_pred_clf_lr_bert_test = clf_lr_BERT.predict(test_embeddings_bert)

accuracy_clf_lr_bert_test = accuracy_score(y_test,y_pred_clf_lr_bert_test)
precision_clf_lr_bert_test = precision_score(y_test,y_pred_clf_lr_bert_test, average='weighted')
recall_clf_lr_bert_test = recall_score(y_test, y_pred_clf_lr_bert_test, average='weighted')
f1_clf_lr_bert_test = f1_score(y_test, y_pred_clf_lr_bert_test, average='weighted')
print(f"validation Accuracy: {accuracy_clf_lr_bert_test}")
print(f"Precision for test: {precision_clf_lr_bert_test:.2f}")
print(f"Recall for test: {recall_clf_lr_bert_test:.2f}")
print(f"F1 Score for test: {f1_clf_lr_bert_test:.2f}")
classification_report_test=classification_report(y_test,y_pred_clf_lr_bert_test)
print(classification_report_test)

bert_results['Logestic Regression Accuracy BERT test']=accuracy_clf_lr_bert_test
bert_results['Logestic Regression Precision BERT test']=precision_clf_lr_bert_test
bert_results['Logestic Regression Recall BERT test']=recall_clf_lr_bert_test
bert_results['Logistic Regression F1 BERT test']=f1_clf_lr_bert_test

validation Accuracy: 0.877
Precision for test: 0.88
Recall for test: 0.88
F1 Score for test: 0.87
              precision    recall  f1-score   support

    economic       0.83      0.74      0.78       200
    politics       0.87      0.94      0.90       512
       sport       0.93      0.97      0.95       200
        tech       0.93      0.58      0.71        88

    accuracy                           0.88      1000
   macro avg       0.89      0.81      0.84      1000
weighted avg       0.88      0.88      0.87      1000



## Neural Networks

#### embedding weights

In [ ]:
# NOTE BERT handles OOV automatically and intializes it with zeros
model_name="aubmindlab/bert-base-arabertv02"
tokenizer_bert = BertTokenizer.from_pretrained(model_name,add_special_tokens=False)
model = BertModel.from_pretrained(model_name)
embedding_weight_matrix_bert=np.zeros((total_words,768))
for word, i in tokenizer.word_index.items():
  tokens=tokenizer_bert(word,return_tensors='pt',add_special_tokens=False)
  input_ids=tokens['input_ids']
  with torch.no_grad():
    embedding_vector_bert=model(input_ids)
  embedding_weight_matrix_bert[i]=embedding_vector_bert.last_hidden_state.mean(dim=1).squeeze().numpy()

### LSTM

In [ ]:
model_LSTM_bert=Sequential()
model_LSTM_bert.add(Embedding(input_dim=total_words,
                    output_dim=train_embeddings_bert.shape[1],
                    input_length=max_sequence_len,
                    weights=[embedding_weight_matrix_bert]))
model_LSTM_bert.add(LSTM(100))
model_LSTM_bert.add(Dense(4,activation='softmax'))
adam=Adam(learning_rate=0.01)
model_LSTM_bert.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['accuracy'])
model_LSTM_bert.fit(padded_train,y_train_one_hot_encoded,epochs=10,verbose=1)
val_loss, val_accuracy = model_LSTM_bert.evaluate(padded_val, y_val_one_hot_encoded, verbose=1)
print(f'Validation Accuracy: {val_accuracy}')

y_pred = model_LSTM_bert.predict(padded_test)
y_pred_LSTM_bert_val= np.argmax(y_pred, axis=1)
accuracy_LSTM_bert_val = accuracy_score(y_test_encoded,y_pred_LSTM_bert_val)
precision_LSTM_bert_val = precision_score(y_test_encoded,y_pred_LSTM_bert_val, average='weighted')
recall_LSTM_bert_val = recall_score(y_test_encoded,y_pred_LSTM_bert_val, average='weighted')
f1_LSTM_bert_val = f1_score(y_test_encoded, y_pred_LSTM_bert_val, average='weighted')
print('Classification report')
print(classification_report(y_test_encoded, y_pred_LSTM_bert_val))
print(f'Accuracy: {accuracy_LSTM_bert_val:.4f}')
print(f'Precision: {precision_LSTM_bert_val:.4f}')
print(f'Recall: {recall_LSTM_bert_val:.4f}')
print(f'F1 Score: {f1_LSTM_bert_val:.4f}')
bert_results['LSTM Accuracy BERT val']=accuracy_LSTM_bert_val
bert_results['LSTM Precision BERT val']=precision_LSTM_bert_val
bert_results['LSTM  Recall BERT val']=recall_LSTM_bert_val
bert_results['LSTM  F1 BERT val']=f1_LSTM_bert_val

Epoch 1/10
125/125 [==============================] - 19s 133ms/step - loss: 1.0396 - accuracy: 0.6122
Epoch 2/10
125/125 [==============================] - 17s 136ms/step - loss: 0.9961 - accuracy: 0.6195
Epoch 3/10
125/125 [==============================] - 17s 133ms/step - loss: 0.9682 - accuracy: 0.6183
Epoch 4/10
125/125 [==============================] - 17s 133ms/step - loss: 0.8180 - accuracy: 0.6668
Epoch 5/10
125/125 [==============================] - 17s 135ms/step - loss: 0.7204 - accuracy: 0.7215
Epoch 6/10
125/125 [==============================] - 17s 137ms/step - loss: 0.5829 - accuracy: 0.7893
Epoch 7/10
125/125 [==============================] - 17s 139ms/step - loss: 0.5045 - accuracy: 0.8210
Epoch 8/10
125/125 [==============================] - 18s 140ms/step - loss: 0.4625 - accuracy: 0.8428
Epoch 9/10
125/125 [==============================] - 18s 146ms/step - loss: 0.4377 - accuracy: 0.8465
Epoch 10/10
32/32 [==============================] - 2s 34ms/step - loss:

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:13

In [ ]:
test_loss_bert_lstm, test_accuracy_bert_lstm = model_LSTM_bert.evaluate(padded_test, y_test_one_hot_encoded, verbose=1)
print(f'test Accuracy: {test_accuracy_bert_lstm}')

y_pred = model_LSTM_bert.predict(padded_test)
y_pred_LSTM_bert_test= np.argmax(y_pred, axis=1)
accuracy_LSTM_bert_test = accuracy_score(y_test_encoded,y_pred_LSTM_bert_test)
precision_LSTM_bert_test = precision_score(y_test_encoded,y_pred_LSTM_bert_test, average='weighted')
recall_LSTM_bert_test = recall_score(y_test_encoded,y_pred_LSTM_bert_test, average='weighted')
f1_LSTM_bert_test = f1_score(y_test_encoded, y_pred_LSTM_bert_test, average='weighted')
print('Classification report')
print(classification_report(y_test_encoded, y_pred_LSTM_bert_test))

print(f'Accuracy: {accuracy_LSTM_bert_test:.4f}')
print(f'Precision: {precision_LSTM_bert_test:.4f}')
print(f'Recall: {recall_LSTM_bert_test:.4f}')
print(f'F1 Score: {f1_LSTM_bert_test:.4f}')
bert_results['LSTM Accuracy BERT test']=accuracy_LSTM_bert_test
bert_results['LSTM Precision BERT test']=precision_LSTM_bert_test
bert_results['LSTM  Recall BERT test']=recall_LSTM_bert_test
bert_results['LSTM  F1 BERT test']=f1_LSTM_bert_test

32/32 [==============================] - 1s 35ms/step - loss: 0.9172 - accuracy: 0.6990
test Accuracy: 0.6990000009536743
32/32 [==============================] - 1s 24ms/step
Classification report
              precision    recall  f1-score   support

           0       0.49      0.46      0.47       200
           1       0.75      0.92      0.83       512
           2       0.75      0.68      0.71       200
           3       0.00      0.00      0.00        88

    accuracy                           0.70      1000
   macro avg       0.50      0.51      0.50      1000
weighted avg       0.63      0.70      0.66      1000

Accuracy: 0.6990
Precision: 0.6297
Recall: 0.6990
F1 Score: 0.6592


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:13

### RNN

In [ ]:
model_rnn_bert=Sequential()
model_rnn_bert.add(Embedding(input_dim=total_words,
                    output_dim=train_embeddings_bert.shape[1],
                    input_length=max_sequence_len,
                    weights=[embedding_weight_matrix_bert]))
model_rnn_bert.add(SimpleRNN(100))
model_rnn_bert.add(Dense(4,activation='softmax'))
adam=Adam(learning_rate=0.01)
model_rnn_bert.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['accuracy'])
model_rnn_bert.fit(padded_train,y_train_one_hot_encoded,epochs=10,verbose=1)
val_loss_bert_rnn, val_accuracy_bert_rnn = model_rnn_bert.evaluate(padded_val, y_val_one_hot_encoded, verbose=1)
print(f'Validation Accuracy: {val_accuracy_bert_rnn}')

y_pred = model_rnn_bert.predict(padded_val)
y_pred_rnn_bert_val= np.argmax(y_pred, axis=1)
accuracy_rnn_bert_val = accuracy_score(y_val_encoded,y_pred_rnn_bert_val)
precision_rnn_bert_val = precision_score(y_val_encoded,y_pred_rnn_bert_val, average='weighted')
recall_rnn_bert_val = recall_score(y_val_encoded,y_pred_rnn_bert_val, average='weighted')
f1_rnn_bert_val = f1_score(y_val_encoded, y_pred_rnn_bert_val, average='weighted')
print('Classification report')
print(classification_report(y_val_encoded, y_pred_rnn_bert_val))
print(f'Accuracy: {accuracy_rnn_bert_val:.4f}')
print(f'Precision: {precision_rnn_bert_val:.4f}')
print(f'Recall: {recall_rnn_bert_val:.4f}')
print(f'F1 Score: {f1_rnn_bert_val:.4f}')
bert_results['RNN Accuracy BERT test']=accuracy_rnn_bert_val
bert_results['RNN Precision BERT test']=precision_rnn_bert_val
bert_results['RNN  Recall BERT test']=recall_rnn_bert_val
bert_results['RNN  F1 BERT test']=f1_rnn_bert_val

Epoch 1/10
125/125 [==============================] - 14s 99ms/step - loss: 1.1001 - accuracy: 0.6150
Epoch 2/10
125/125 [==============================] - 11s 85ms/step - loss: 1.0301 - accuracy: 0.6198
Epoch 3/10
125/125 [==============================] - 12s 97ms/step - loss: 1.0162 - accuracy: 0.6168
Epoch 4/10
125/125 [==============================] - 11s 92ms/step - loss: 1.0346 - accuracy: 0.6205
Epoch 5/10
125/125 [==============================] - 12s 95ms/step - loss: 1.0628 - accuracy: 0.6220
Epoch 6/10
125/125 [==============================] - 11s 88ms/step - loss: 1.0165 - accuracy: 0.6102
Epoch 7/10
125/125 [==============================] - 13s 100ms/step - loss: 0.8990 - accuracy: 0.6488
Epoch 8/10
125/125 [==============================] - 12s 97ms/step - loss: 0.8492 - accuracy: 0.6635
Epoch 9/10
125/125 [==============================] - 12s 98ms/step - loss: 0.8132 - accuracy: 0.6865
Epoch 10/10
32/32 [==============================] - 1s 17ms/step - loss: 0.8874 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:13

In [ ]:
test_loss_bert_rnn, test_accuracy_bert_rnn = model_rnn_bert.evaluate(padded_test, y_test_one_hot_encoded, verbose=1)
print(f'test Accuracy: {test_accuracy_bert_rnn}')

y_pred =model_rnn_bert.predict(padded_test)
y_pred_rnn_bert_test= np.argmax(y_pred, axis=1)
accuracy_rnn_bert_test = accuracy_score(y_test_encoded, y_pred_rnn_bert_test)
precision_rnn_bert_test = precision_score(y_test_encoded, y_pred_rnn_bert_test, average='weighted')
recall_rnn_bert_test = recall_score(y_test_encoded, y_pred_rnn_bert_test, average='weighted')
f1_rnn_bert_test = f1_score(y_test_encoded, y_pred_rnn_bert_test, average='weighted')
print('Classification report')
print(classification_report(y_test_encoded, y_pred_rnn_bert_test))
print(f'Accuracy: {accuracy_rnn_bert_test:.4f}')
print(f'Precision: {precision_rnn_bert_test:.4f}')
print(f'Recall: {recall_rnn_bert_test:.4f}')
print(f'F1 Score: {f1_rnn_bert_test:.4f}')
bert_results['RNN Accuracy BERT val']=accuracy_rnn_bert_test
bert_results['RNN Precision BERT val']=precision_rnn_bert_test
bert_results['RNN  Recall BERT val']=recall_rnn_bert_test
bert_results['RNN  F1 BERT val']=f1_rnn_bert_test

32/32 [==============================] - 1s 16ms/step - loss: 1.2137 - accuracy: 0.5630
test Accuracy: 0.5630000233650208
32/32 [==============================] - 1s 16ms/step
Classification report
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       200
           1       0.64      0.84      0.72       512
           2       0.41      0.67      0.51       200
           3       0.00      0.00      0.00        88

    accuracy                           0.56      1000
   macro avg       0.26      0.38      0.31      1000
weighted avg       0.41      0.56      0.47      1000

Accuracy: 0.5630
Precision: 0.4083
Recall: 0.5630
F1 Score: 0.4724


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:13

## BERT sequence classifier

In [ ]:
train_labels = torch.tensor(y_train_encoded).long()
val_labels = torch.tensor(y_val_encoded).long()
test_labels = torch.tensor(y_test_encoded).long()

In [ ]:
model_name = "aubmindlab/bert-base-arabertv02"
tokenizer = BertTokenizer.from_pretrained(model_name)
bert_model = BertModel.from_pretrained(model_name)
bert_clf = BertForSequenceClassification.from_pretrained(model_name, num_labels=4)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv02 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
tokens_train=tokenizer(train_df['processed_text'].tolist(),return_tensors='pt',padding=True,truncation=True)


input_ids_train=tokens_train['input_ids']
attention_mask_train=tokens_train['attention_mask']


In [ ]:
tokens_val=tokenizer(val_df['processed_text'].tolist(),return_tensors='pt',padding=True,truncation=True)
input_ids_val=tokens_val['input_ids']
attention_mask_val=tokens_val['attention_mask']

In [ ]:
tokens_test=tokenizer(df_test['processed_text'].tolist(),return_tensors='pt',padding=True,truncation=True)
input_ids_test=tokens_test['input_ids']
attention_mask_test=tokens_test['attention_mask']

In [ ]:
optimizer = AdamW(bert_clf.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()
batch_size = 8

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
bert_clf.train()
total_loss = 0
for i in range(0, len(input_ids_train), batch_size):
  batch_input_ids = input_ids_train[i:i+batch_size]
  batch_attention_mask = attention_mask_train[i:i+batch_size]
  batch_labels = train_labels[i:i+batch_size]
  optimizer.zero_grad()
  outputs = bert_clf(input_ids=batch_input_ids, attention_mask=batch_attention_mask, labels=batch_labels)
  loss = outputs.loss
  total_loss += loss.item()
  loss.backward()
  optimizer.step()

avg_train_loss = total_loss / (len(input_ids_train) // batch_size)
# Validation
bert_clf.eval()
predictions, true_labels = [], []
with torch.no_grad():
  for i in range(0, len(input_ids_val), batch_size):
    batch_input_ids = input_ids_val[i:i+batch_size]
    batch_attention_mask = attention_mask_val[i:i+batch_size]
    batch_labels = val_labels[i:i+batch_size]
    outputs = bert_clf(input_ids=batch_input_ids, attention_mask=batch_attention_mask)
    logits = outputs.logits
    preds = torch.argmax(logits, dim=-1)
    predictions.extend(preds.numpy())
    true_labels.extend(batch_labels.numpy())

val_accuracy = accuracy_score(true_labels, predictions)
val_precision = precision_score(true_labels, predictions, average='weighted')
val_recall = recall_score(true_labels, predictions, average='weighted')
val_f1 = f1_score(true_labels, predictions, average='weighted')

print(f'Validation Accuracy: {val_accuracy * 100:.2f}%')
print(f'Validation Precision: {val_precision * 100:.2f}%')
print(f'Validation Recall: {val_recall * 100:.2f}%')
print(f'Validation F1 Score: {val_f1 * 100:.2f}%')
print(classification_report(true_labels, predictions))
bert_results['BERT Squence classifier Accuracy BERT val']=val_accuracy
bert_results['BERT Squence classifier Precision BERT val']=val_precision
bert_results['BERT Squence classifier  Recall BERT val']=val_recall
bert_results['BERT Squence classifier F1 BERT val']=val_f1

# Testing
test_predictions, test_true_labels = [], []
with torch.no_grad():
    for i in range(0, len(input_ids_test), batch_size):
        batch_input_ids = input_ids_test[i:i+batch_size]
        batch_attention_mask = attention_mask_test[i:i+batch_size]
        batch_labels = test_labels[i:i+batch_size]

        outputs = bert_clf(input_ids=batch_input_ids, attention_mask=batch_attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1)

        test_predictions.extend(preds.numpy())
        test_true_labels.extend(batch_labels.numpy())

# Calculate test metrics
test_accuracy = accuracy_score(test_true_labels, test_predictions)
test_precision = precision_score(test_true_labels, test_predictions, average='weighted')
test_recall = recall_score(test_true_labels, test_predictions, average='weighted')
test_f1 = f1_score(test_true_labels, test_predictions, average='weighted')

print(f'Test Accuracy: {test_accuracy * 100:.2f}%')
print(f'Test Precision: {test_precision * 100:.2f}%')
print(f'Test Recall: {test_recall * 100:.2f}%')
print(f'Test F1 Score: {test_f1 * 100:.2f}%')
print(classification_report(test_true_labels, test_predictions, target_names=label.classes_))
bert_results['BERT Squence classifier Accuracy BERT test']=test_accuracy
bert_results['BERT Squence classifier Precision BERT test']=test_precision
bert_results['BERT Squence classifier  Recall BERT test']=test_recall
bert_results['BERT Squence classifier F1 BERT test']=test_f1



Validation Accuracy: 94.50%
Validation Precision: 94.39%
Validation Recall: 94.50%
Validation F1 Score: 94.43%
              precision    recall  f1-score   support

           0       0.90      0.87      0.89       180
           1       0.96      0.96      0.96       601
           2       0.96      0.99      0.98       198
           3       0.72      0.62      0.67        21

    accuracy                           0.94      1000
   macro avg       0.89      0.86      0.87      1000
weighted avg       0.94      0.94      0.94      1000

Test Accuracy: 88.90%
Test Precision: 88.73%
Test Recall: 88.90%
Test F1 Score: 88.69%
              precision    recall  f1-score   support

    economic       0.82      0.80      0.81       200
    politics       0.90      0.93      0.91       512
       sport       0.94      0.98      0.96       200
        tech       0.86      0.67      0.75        88

    accuracy                           0.89      1000
   macro avg       0.88      0.84      0.

## Evalution of BERT across models

In [ ]:
bert_results_list=[{'performance metric & model':k,"results":v} for k,v in bert_results.items()]
bert_df_results=pd.DataFrame(bert_results_list)
bert_df_results

,performance metric & model,results
0,Naive Bayes Accuracy BERT val,0.916000
1,Naive Bayes Precision BERT val,0.922863
2,Naive Bayes Recall BERT val,0.916000
3,Naive Bayes F1 BERT val,0.917976
4,Naive Bayes Accuracy BERT test,0.893000
5,Naive Bayes Precision BERT test,0.892332
6,Naive Bayes Recall BERT test,0.893000
7,Naive Bayes F1 BERT test,0.891857
8,Logestic Regression Accuracy BERT val,0.922000
9,Logestic Regression Precision BERT val,0.921997


# GPT

In [ ]:
gpt_results={}

In [ ]:
model_name = "aubmindlab/aragpt2-base"
tokenizer_gpt = GPT2Tokenizer.from_pretrained(model_name)
gpt_clf = GPT2ForSequenceClassification.from_pretrained(model_name, num_labels=4)
tokenizer_gpt.pad_token = tokenizer_gpt.eos_token
gpt_clf.config.pad_token_id = tokenizer_gpt.pad_token_id

vocab.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.50M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.52M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/553M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at aubmindlab/aragpt2-base and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
optimizer = AdamW(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()



/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
train_texts = train_df['processed_text'].tolist()
val_texts = val_df['processed_text'].tolist()
test_texts = df_test['processed_text'].tolist()

train_encodings_gpt = tokenizer_gpt(train_texts, padding=True, return_tensors='pt')
val_encodings_gpt = tokenizer_gpt(val_texts, truncation=True, padding=True,return_tensors='pt')
test_encodings_gpt = tokenizer_gpt(test_texts, truncation=True, padding=True,return_tensors='pt')

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)
test_labels = torch.tensor(test_labels)

<ipython-input-106-581dd3cf2fa5>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels = torch.tensor(train_labels)
<ipython-input-106-581dd3cf2fa5>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  val_labels = torch.tensor(val_labels)
<ipython-input-106-581dd3cf2fa5>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_labels = torch.tensor(test_labels)


## GPT sequence classifier

In [ ]:
batch_size = 8


gpt_clf.train()
total_loss = 0
for i in range(0, len(train_encodings_gpt['input_ids']), batch_size):
  batch_input_ids = train_encodings_gpt['input_ids'][i:i+batch_size]
  batch_attention_mask = train_encodings_gpt['attention_mask'][i:i+batch_size]
  batch_labels = train_labels[i:i+batch_size]
  optimizer.zero_grad()
  outputs = gpt_clf(input_ids=batch_input_ids, attention_mask=batch_attention_mask, labels=batch_labels)
  loss = outputs.loss
  total_loss += loss.item()
  loss.backward()
  optimizer.step()




In [ ]:
# Validation
gpt_clf.eval()
predictions, true_labels = [], []
with torch.no_grad():
  for i in range(0, len(val_encodings_gpt['input_ids']), batch_size):
    batch_input_ids = val_encodings_gpt['input_ids'][i:i+batch_size]
    batch_attention_mask = val_encodings_gpt['attention_mask'][i:i+batch_size]
    batch_labels = val_labels[i:i+batch_size]

    outputs = gpt_clf(input_ids=batch_input_ids, attention_mask=batch_attention_mask)
    logits = outputs.logits
    preds = torch.argmax(logits, dim=-1)

    predictions.extend(preds.numpy())
    true_labels.extend(batch_labels.numpy())

val_accuracy = accuracy_score(true_labels, predictions)
val_precision = precision_score(true_labels, predictions, average='weighted')
val_recall = recall_score(true_labels, predictions, average='weighted')
val_f1 = f1_score(true_labels, predictions, average='weighted')

print(f'Validation Accuracy: {val_accuracy * 100:.2f}%')
print(f'Validation Precision: {val_precision * 100:.2f}%')
print(f'Validation Recall: {val_recall * 100:.2f}%')
print(f'Validation F1 Score: {val_f1 * 100:.2f}%')
print(classification_report(true_labels, predictions))
gpt_results['GPT Squence classifier Accuracy GPT val']=val_accuracy
gpt_results['GPT Squence classifier Precision GPT val']=val_precision
gpt_results['GPT Squence classifier  Recall GPT val']=val_recall
gpt_results['GPT Squence classifier F1 GPT val']=val_f1



Validation Accuracy: 14.70%
Validation Precision: 40.35%
Validation Recall: 14.70%
Validation F1 Score: 13.74%
              precision    recall  f1-score   support

           0       0.10      0.20      0.14       180
           1       0.57      0.05      0.10       601
           2       0.21      0.40      0.27       198
           3       0.00      0.00      0.00        21

    accuracy                           0.15      1000
   macro avg       0.22      0.16      0.13      1000
weighted avg       0.40      0.15      0.14      1000



In [ ]:
# Testing
test_predictions, test_true_labels = [], []
with torch.no_grad():
    for i in range(0, len(test_encodings_gpt['input_ids']), batch_size):
        batch_input_ids = test_encodings_gpt['input_ids'][i:i+batch_size]
        batch_attention_mask = test_encodings_gpt['attention_mask'][i:i+batch_size]
        batch_labels = test_labels[i:i+batch_size]

        outputs = gpt_clf(input_ids=batch_input_ids, attention_mask=batch_attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1)

        test_predictions.extend(preds.numpy())
        test_true_labels.extend(batch_labels.numpy())

test_accuracy = accuracy_score(test_true_labels, test_predictions)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')
print(classification_report(test_true_labels, test_predictions, target_names=label.classes_))

test_accuracy = accuracy_score(test_true_labels, test_predictions)
test_precision = precision_score(test_true_labels, test_predictions, average='weighted')
test_recall = recall_score(test_true_labels, test_predictions, average='weighted')
test_f1 = f1_score(test_true_labels, test_predictions, average='weighted')

print(f'Test Accuracy: {test_accuracy * 100:.2f}%')
print(f'Test Precision: {test_precision * 100:.2f}%')
print(f'Test Recall: {test_recall * 100:.2f}%')
print(f'Test F1 Score: {test_f1 * 100:.2f}%')
gpt_results['GPT Squence classifier Accuracy GPT test']=test_accuracy
gpt_results['GPT Squence classifier Precision GPT test']=test_precision
gpt_results['GPT Squence classifier  Recall GPT test']=test_recall
gpt_results['GPT Squence classifier F1 GPT test']=test_f1

Test Accuracy: 13.40%
              precision    recall  f1-score   support

    economic       0.09      0.15      0.12       200
    politics       0.58      0.07      0.13       512
       sport       0.15      0.28      0.20       200
        tech       0.04      0.09      0.05        88

    accuracy                           0.13      1000
   macro avg       0.22      0.15      0.12      1000
weighted avg       0.35      0.13      0.13      1000

Test Accuracy: 13.40%
Test Precision: 35.11%
Test Recall: 13.40%
Test F1 Score: 13.45%


## Evaluation of GPT

In [ ]:
gpt_results_list=[{'performance metric & model':k,"results":v} for k,v in gpt_results.items()]
gpt_df_results=pd.DataFrame(gpt_results_list)

In [ ]:
gpt_df_results

,performance metric & model,results
0,GPT Squence classifier Accuracy GPT val,0.147000
1,GPT Squence classifier Precision GPT val,0.403483
2,GPT Squence classifier Recall GPT val,0.147000
3,GPT Squence classifier F1 GPT val,0.137449
4,GPT Squence classifier Accuracy GPT test,0.134000
5,GPT Squence classifier Precision GPT test,0.351071
6,GPT Squence classifier Recall GPT test,0.134000
7,GPT Squence classifier F1 GPT test,0.134526


In [ ]:
df_results=pd.concat([bert_df_results,gpt_df_results],axis=0)

In [ ]:
df_results

,performance metric & model,results
0,Naive Bayes Accuracy BERT val,0.916000
1,Naive Bayes Precision BERT val,0.922863
2,Naive Bayes Recall BERT val,0.916000
3,Naive Bayes F1 BERT val,0.917976
4,Naive Bayes Accuracy BERT test,0.893000
5,Naive Bayes Precision BERT test,0.892332
6,Naive Bayes Recall BERT test,0.893000
7,Naive Bayes F1 BERT test,0.891857
8,Logestic Regression Accuracy BERT val,0.922000
9,Logestic Regression Precision BERT val,0.921997


In [ ]:
df_results.to_csv("bert & gpt evaluation3.csv",index=False)